In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
data = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=88000)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
word_index = data.get_word_index()
word_index = {k: v+3 for (k, v) in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3
reversed_word_index = {v: k for (k, v) in word_index.items()}

1654784/1641221 [==============================] - 0s 0us/step


In [5]:
def decode_review(text):
  return " ".join(reversed_word_index.get(word, "?") for word in text)

In [6]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], maxlen=250, padding="post")
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], maxlen=250, padding="post")

In [7]:
# Model
model = keras.Sequential()
model.add(keras.layers.Embedding(88000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [8]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [9]:
# Validation Data
x_val = train_data[:10000]
x_train = train_data[10000:]

y_val = train_labels[:10000]
y_train = train_labels[10000:]

In [10]:
model.fit(x_train, y_train, batch_size=512, epochs=40, validation_data=(x_val, y_val), verbose=1)

Epoch 1/40
30/30 [==============================] - 2s 47ms/step - loss: 0.6914 - accuracy: 0.5218 - val_loss: 0.6890 - val_accuracy: 0.6348
Epoch 2/40
30/30 [==============================] - 1s 40ms/step - loss: 0.6845 - accuracy: 0.7072 - val_loss: 0.6802 - val_accuracy: 0.7135
Epoch 3/40
30/30 [==============================] - 1s 39ms/step - loss: 0.6708 - accuracy: 0.7521 - val_loss: 0.6641 - val_accuracy: 0.7509
Epoch 4/40
30/30 [==============================] - 1s 38ms/step - loss: 0.6475 - accuracy: 0.7799 - val_loss: 0.6383 - val_accuracy: 0.7743
Epoch 5/40
30/30 [==============================] - 1s 39ms/step - loss: 0.6126 - accuracy: 0.8063 - val_loss: 0.6036 - val_accuracy: 0.7935
Epoch 6/40
30/30 [==============================] - 1s 39ms/step - loss: 0.5688 - accuracy: 0.8287 - val_loss: 0.5626 - val_accuracy: 0.8073
Epoch 7/40
30/30 [==============================] - 1s 39ms/step - loss: 0.5194 - accuracy: 0.8454 - val_loss: 0.5202 - val_accuracy: 0.8221
Epoch 8/40
30

In [11]:
results = model.evaluate(test_data, test_labels)

782/782 [==============================] - 2s 2ms/step - loss: 0.3344 - accuracy: 0.8726


In [12]:
model.save("reviews_classifier.h5")

In [13]:
classifier = keras.models.load_model("reviews_classifier.h5")

In [14]:
from re import subn
def encode_review(text):
  rep = "[,.:/\\|()+-=*!@#$%^&]"
  text = subn(rep, "", text)[0].lower().split(' ')
  review = [1]
  for word in text:
    review.append(word_index.get(word, 2))
  
  review = keras.preprocessing.sequence.pad_sequences([review], padding="post", value=0, maxlen=250)
  return review

In [15]:
review = "this movie was great"
review = encode_review(review)
print(review)

[[ 1 14 20 16 87  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]]


In [16]:
pred = classifier.predict(review)

In [17]:
print(pred[0][0])

0.72413015
